In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\wine_quality\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\DataScience\\Projects\\wine_quality'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [8]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml_file, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        self.schema = read_yaml_file(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [11]:
import pandas as pd
import os
from mlproject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_model_trainer_config()
    data_transformation = ModelTrainer(config=data_transformation_config)
    data_transformation.train()
except Exception as e:
    raise e

[2024-02-23 17:36:39,711: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-02-23 17:36:39,722: INFO:common: yaml file: params.yaml loaded successfully]
[2024-02-23 17:36:39,727: INFO:common: yaml file: schema.yaml loaded successfully]
[2024-02-23 17:36:39,731: INFO:common: createD Directory at:artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'model_trainer'"